## Model Tests if Working or not

In [3]:
import os
import sys

sys.path.append(os.path.abspath("../src"))

In [66]:
from repoai.agent.gemini_agent import make_gemini_agent

In [67]:
agent = make_gemini_agent()
response = await agent.run("Hello, Gemini! Can you respond to me?")
print(response.output)

Hello! Yes, I can respond to you. How can I help you today?


In [68]:
aiml_agent = make_aimlapi_agent()
aiml_repsonse = await aiml_agent.run("Hello, AIMLAPI! Can you respond to me?")
print(aiml_repsonse.output)

Hello! Yes, I'm here and ready to respond. How can I assist you today?


### Testing if the defaults work or not

In [3]:
from repoai.llm import load_defaults_from_env, ModelRole

In [4]:
# Load default model specs from environment variables
table = load_defaults_from_env()

for role, specs in table.items():
    print(f"{role}: {[s.model_id for s in specs]}")

ModelRole.INTAKE: ['deepseek/deepseek-chat-v3.1', 'alibaba/qwen-max', 'claude-sonnet-4-5-20250929']
ModelRole.PLANNER: ['deepseek/deepseek-reasoner-v3.1', 'alibaba/qwen3-next-80b-a3b-thinking', 'claude-opus-4-20250514']
ModelRole.PR_NARRATOR: ['deepseek/deepseek-chat-v3.1', 'claude-haiku-4-5-20251001', 'alibaba/qwen3-235b-a22b-thinking-2507']
ModelRole.CODER: ['alibaba/qwen3-coder-480b-a35b-instruct', 'Qwen/Qwen2.5-Coder-32B-Instruct', 'deepseek/deepseek-chat-v3.1', 'claude-opus-4-1-20250805']
ModelRole.EMBEDDING: ['bge-small']


### AIML Client Wrapper

In [12]:
import os
import sys, json

sys.path.append(os.path.abspath("../src"))

In [24]:
# Reload to pick up AIMLAPI_API_KEY fix
import importlib
import sys

# Clear cached modules
for mod_name in list(sys.modules.keys()):
    if 'aiml_client' in mod_name or mod_name.startswith('repoai.llm.clients'):
        del sys.modules[mod_name]

from dotenv import load_dotenv
load_dotenv('/home/timmy/RepoAI_AI/.env', override=True)

from repoai.llm.clients.aiml_client import AIMLClient
print("✓ AIMLClient reloaded with AIMLAPI_API_KEY fix")

✓ AIMLClient reloaded with AIMLAPI_API_KEY fix


In [25]:
client = AIMLClient()

message = [
    {"role": "system", "content": "You are RepoAI, an AI code assistant."},
    {"role": "user", "content": "Say hello from AIML."}
]

response = client.chat("deepseek/deepseek-chat-v3.1", message)
print(json.dumps(response, indent=2))

{
  "id": "14b97720-3a9f-4131-ac92-ccd41f26dac6",
  "system_fingerprint": "fp_ffc7281d48_prod0820_fp8_kvcache",
  "object": "chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null,
      "message": {
        "role": "assistant",
        "content": "Hello from AIML! How can I assist you with your code today?",
        "reasoning_content": ""
      }
    }
  ],
  "created": 1761147813,
  "model": "deepseek-chat",
  "usage": {
    "prompt_tokens": 25,
    "completion_tokens": 53,
    "total_tokens": 78,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "prompt_cache_hit_tokens": 0,
    "prompt_cache_miss_tokens": 21
  }
}


In [17]:
response["choices"][0]["message"]["content"]

'Hello from AIML! How can I assist you with your code today?'

### Making use of Model Router

In [37]:
import sys, json
import os
sys.path.append(os.path.abspath("../src"))

In [47]:
# Reload all llm modules to pick up max_tokens fix
import sys
modules_to_clear = [m for m in list(sys.modules.keys()) if m.startswith('repoai.llm')]
for m in modules_to_clear:
    del sys.modules[m]

from dotenv import load_dotenv
load_dotenv('/home/timmy/RepoAI_AI/.env', override=True)

from repoai.llm import ModelRouter, ModelRole

# Create fresh router with updated code
router = ModelRouter()
print("✓ ModelRouter created with max_tokens fix")

✓ ModelRouter created with max_tokens fix


In [39]:
router = ModelRouter()

In [48]:
#inspect chosen model IDs (From Defaults)
for role in [ModelRole.INTAKE, ModelRole.PLANNER, ModelRole.CODER]:
    client = router.choose(role)
    print(f"Chosen model for {role}: {client.model_id}")

Chosen model for ModelRole.INTAKE: deepseek/deepseek-chat-v3.1
Chosen model for ModelRole.PLANNER: deepseek/deepseek-reasoner-v3.1
Chosen model for ModelRole.CODER: alibaba/qwen3-coder-480b-a35b-instruct


In [52]:
# Test router.chat_with_fallback() - explicitly set json_mode=False
messages = [
    {"role": "system", "content": "You are RepoAI, an AI code assistant."},
    {"role": "user", "content": "Say hello from AIML."}
]

resp = router.chat_with_fallback(ModelRole.INTAKE, messages, max_output_tokens=50, json_mode=False)
print(json.dumps(resp, indent=2))

Payload being sent: {'model': 'deepseek/deepseek-chat-v3.1', 'messages': [{'role': 'system', 'content': 'You are RepoAI, an AI code assistant.'}, {'role': 'user', 'content': 'Say hello from AIML.'}], 'temperature': 0.3, 'max_tokens': 50}
{
  "id": "6ace4e7c-26c4-482f-9857-fb9135ab3966",
  "system_fingerprint": "fp_ffc7281d48_prod0820_fp8_kvcache",
  "object": "chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null,
      "message": {
        "role": "assistant",
        "content": "Hello from AIML! How can I assist you with your code today?",
        "reasoning_content": ""
      }
    }
  ],
  "created": 1761150393,
  "model": "deepseek-chat",
  "usage": {
    "prompt_tokens": 25,
    "completion_tokens": 53,
    "total_tokens": 78,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "prompt_cache_hit_tokens": 0,
    "prompt_cache_miss_tokens": 21
  }
}
{
  "id": "6ace4e7c-26c4-482f-9857-fb9135ab3966",
  "system_fi

In [53]:
response["choices"][0]["message"]["content"]

'Hello from AIML! How can I assist you with your code today?'